In [ ]:
import numpy as np
import tensorflow as tf
import keras
from model import *
import datetime
from utils import *
from preprocessing import *
import random
import glob
from os.path import join, getctime, basename
from load_data import load_data
from save_model import replace_model_if_better
from keras.models import load_model
from shutil import rmtree

In [ ]:
# Create result folders 
save_path = join(get_save_path(), 'SimpleConvGRU')
new_path = join(save_path, 'best_val')
time = str(datetime.datetime.now()).replace(' ', '_')
workpath = join(save_path, time)
ensure_dir(new_path)
ensure_dir(workpath)
ensure_dir(save_path)

# Load preprocessed data
aug_data = load_data_pickle(get_save_path())
total_X_test = aug_data['total_X_test']
total_y_test = aug_data['total_y_test']
total_X_train = aug_data['total_X_train']
total_y_train = aug_data['total_y_train']
total_X_val = aug_data['total_X_val']
total_y_val = aug_data['total_y_val']
total_X_train = np.transpose(total_X_train, (0, 2, 1))
total_X_val = np.transpose(total_X_val, (0, 2, 1))
total_X_test = np.transpose(total_X_test, (0, 2, 1))
print(total_X_train.shape)
print(total_y_train.shape)
print(total_X_test.shape)
print(total_y_test.shape)
print(total_X_val.shape)
print(total_y_val.shape)

In [1]:
config = {
    # Network
    'num_inputs': total_X_train.shape[0],
    'input_shape': (total_X_train.shape[1],total_X_train.shape[2],1),
    'epochs': 50,
    'dropout': 0.3,
    'batch_size': 320,
    'l2': 0.01,
    'lr': 0.001
}

Using TensorFlow backend.


In [2]:
# Create result folders 
save_path = join(get_save_path(), 'SimpleConvGRU')
new_path = join(save_path, 'best_val')
time = str(datetime.datetime.now()).replace(' ', '_')
workpath = join(save_path, time)
ensure_dir(new_path)
ensure_dir(workpath)
ensure_dir(save_path)

# Load preprocessed data
aug_data = load_data_pickle(get_save_path())
total_X_test = aug_data['total_X_test']
total_y_test = aug_data['total_y_test']
total_X_train = aug_data['total_X_train']
total_y_train = aug_data['total_y_train']
total_X_val = aug_data['total_X_val']
total_y_val = aug_data['total_y_val']
total_X_train = np.transpose(total_X_train, (0, 2, 1))
total_X_val = np.transpose(total_X_val, (0, 2, 1))
total_X_test = np.transpose(total_X_test, (0, 2, 1))
print(total_X_train.shape)
print(total_y_train.shape)
print(total_X_test.shape)
print(total_y_test.shape)
print(total_X_val.shape)
print(total_y_val.shape)

Loading data pickle...
Data pickle loaded.
(11844, 100, 22)
(11844, 4)
(3101, 100, 22)
(3101, 4)
(2961, 100, 22)
(2961, 4)


In [3]:
#total_X_train, total_y_train = shuffle(total_X_train, total_y_train, random_state=random.randint(1,1000))
#total_X_val, total_y_val = shuffle(total_X_val, total_y_val, random_state=random.randint(1,1000))
# total_X_test, total_y_test = shuffle(total_X_test, total_y_test, random_state=random.randint(1,1000))
total_X_train = np.transpose(total_X_train, (0, 2, 1))
total_X_val = np.transpose(total_X_val, (0, 2, 1))
total_X_test = np.transpose(total_X_test, (0, 2, 1))
print(total_X_train.shape)
print(total_y_train.shape)
print(total_X_test.shape)
print(total_y_test.shape)
print(total_X_val.shape)
print(total_y_val.shape)

(11844, 100, 22)
(11844, 4)
(3101, 100, 22)
(3101, 4)
(2961, 100, 22)
(2961, 4)


In [3]:
config = {
    # Network
    'num_inputs': total_X_train.shape[0],
    'input_shape': (total_X_train.shape[1],total_X_train.shape[2],1),
    'epochs': 50,
    'dropout': 0.3,
    'batch_size': 320,
    'l2': 0.01,
    'lr': 0.001
}

In [4]:
SimpleGRUConv = SimpleGRUConv()
SimpleGRUConv.build_model(config)
SimpleGRUConv.train(total_X_train, total_y_train, total_X_val, total_y_val, config, workpath)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 100, 22)           2970      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 91, 22)            4862      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2002)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2002)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8012      
Total params: 15,844
Trainable params: 15,844
Non-trainable params: 0
_________________________________________________________________
Model compiled.
Train on 11844 samples, validate on 2961 samples
Epoch 1/50
11844/11844 [===========================

Epoch 49/50
11844/11844 [==============================] - 2s 130us/step - loss: 0.2187 - accuracy: 0.9165 - val_loss: 1.6238 - val_accuracy: 0.5684
Epoch 50/50
11844/11844 [==============================] - 2s 130us/step - loss: 0.1726 - accuracy: 0.9341 - val_loss: 1.6584 - val_accuracy: 0.5772


In [5]:
preds = [SimpleGRUConv.predict(total_X_test[i::total_X_test.shape[0]], 0) for i in range(total_X_test.shape[0])]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:total_X_test.shape[0]])]
raw = SimpleGRUConv.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'SimpleGRUConv.pickle')
replace_model_if_better(filepath, np.mean(result), SimpleGRUConv, config)

3101/3101 [==============================] - 0s 130us/step
Raw Acc result: 0.5994840264320374
Majority Vote result: 0.6162528216704289
Old model exists. Comparing performance.
New model is worse than the old one. Will not update the old model


False

In [6]:
model_path = join(workpath, '*.hdf5')
list_of_files = glob.glob(model_path) # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=getctime)
model_val = load_model(latest_file)
preds = [model_val.predict(total_X_test[i::total_X_test.shape[0]], verbose=0) for i in range(total_X_test.shape[0])]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:total_X_test.shape[0]])]
raw = model_val.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'best_val', 'SimpleGRUConv_val.pickle')
replaced = replace_model_if_better(filepath, np.mean(result), model_val, config)
rmtree(workpath)


3101/3101 [==============================] - 0s 145us/step
Raw Acc result: 0.5994840264320374
Majority Vote result: 0.6162528216704289
Old model exists. Comparing performance.
New model is worse than the old one. Will not update the old model
